In [1]:
# A notebook to evaluate GNN explainer performance by calculating Comprehensiveness and Sufficiency 
#     as defined in "BAGEL: A Benchmark for Assessing Graph Neural Network Explanations", by Rathee et al. (2022). 
# 
#     Model training and explanation generation adapted from the original MolRep code by Tim Stols.


In [2]:
# Converted by Tim Stols
# python Explainer_Experiments.py --model_name CMPNN \
#                                                        --attribution_name GradInput \
#                                                        --data_path ../MolRep/Datasets/Metabolism/admet2.1_rlm_merge.csv \
#                                                        --dataset_name RLM \
#                                                        --smiles_col COMPOUND_SMILES \
#                                                        --target_col CLF_LABEL \
#                                                        --task_type Multi-Classification \
#                                                        --multiclass_num_classes 3 \
#                                                        --output_dir ../Outputs

In [3]:
import gc
import os

from pathlib import Path

LOADING_FROM_FILES = True # True if loading in data from pre-existing files

# output directory for trained model and explanations
OUTPUT_DIR = f'../Outputs/Experiment'
# OUTPUT_DIR = '../Outputs' # original output path, overwrites results

# output the explanations (optionally model) here
output_dir = Path(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

DATASET = 'Benzene'
# DATASET = '3MR'

# GNN
MODEL = 'CMPNN' # Options:  ['MPNN', 'DMPNN', 'CMPNN', 'GIN', 'ECC', 'GAT', 'DGCNN', 'DiffPool', 'GraphSAGE', 'GraphNet']

# use specified model, else train a new one
# MODELPATH = None
MODELPATH = '../Outputs/Experiment/CMPNN_Benzene_explainer/CMPNN_1_epoch.pt' # using this model because it has non-zero F1 score
# MODELPATH = '../Models/CMPNN_3MR_explainer_20240519_143138.pt'

if MODELPATH is None:
    MODELPATH = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", f"{MODEL}.pt")

# explainers used on the trained model, overwritten down below for testing purposes
ATTRIBUTIONS = ['IG', 'Random'] # Options: IG CAM MCTS GradInput GradCAM RandomBaseline # this says RandomBaseLine but code expects 'Random'

DATAPATH = '../Datasets/XAI/Benzene/benzene_smiles.csv'
# DATAPATH = '../DataSets/3MR/toy_label_mw350.csv'
SMILESCOL = 'SMILES'
TARGETCOL = 'label'
# TARGETCOL = 'label_full' # for 3MR, on my machine the functions targets the wrong label, even if the offending column is deleted

ATTRIBUTIONPATH = '../Datasets/XAI/Benzene/attributions.npz'
# ATTRIBUTIONPATH = '../DataSets/3MR/attributions.npz'


TASKTYPE = 'Classification' # Can be 'Multi-Classification', 'Classification', 'Regression'
MULTICLASS_NUM_CLASSES = 2 # Can be 3
TESTING = True
SPLITTYPE = 'defined' # Can be 'random', 'scaffold', or other defined for dataset


In [4]:
import rdkit.Chem
from MolRep.Utils.logger import Logger
from MolRep.Explainer.explainerExperiments import ExplainerExperiments
from MolRep.Explainer.explainerDataWrapper import ExplainerDatasetWrapper
from MolRep.Utils.config_from_dict import Grid, Config, DatasetConfig
from pathlib import Path
import os
# from numba.cuda import args

# torch.set_num_threads(1)

# set up the model
data_stats = {
            'name': DATASET,
            'path': DATAPATH,
            'smiles_column': SMILESCOL,
            'target_columns': [TARGETCOL],
            'attribution_path': ATTRIBUTIONPATH,
            'task_type': TASKTYPE,
            'multiclass_num_classes': MULTICLASS_NUM_CLASSES,
            'metric_type': 'rmse' if TASKTYPE == 'Regression' else ['acc', 'auc', 'f1', 'precision', 'recall'],
            'split_type': SPLITTYPE
}

if TESTING:
    data_stats['additional_info'] = {"splits":'SPLIT'}
    
# output for vector groups
data_dir = Path('../MolRep/Data')
split_dir = Path('../MolRep/Splits')
os.makedirs(data_dir, exist_ok=True)
os.makedirs(split_dir, exist_ok=True)

In [5]:
config_file = '../MolRep/Configs/config_{}.yml'.format(MODEL)
model_configurations = Grid(config_file)
model_configuration = Config(**model_configurations[0])
dataset_configuration = DatasetConfig(DATASET, data_dict=data_stats)

CMPNN
{'GIN': <class 'MolRep.Models.graph_based.GIN.GIN'>, 'ECC': <class 'MolRep.Models.graph_based.ECC.ECC'>, 'DGCNN': <class 'MolRep.Models.graph_based.DGCNN.DGCNN'>, 'DiffPool': <class 'MolRep.Models.graph_based.DiffPool.DiffPool'>, 'GraphSAGE': <class 'MolRep.Models.graph_based.GraphSAGE.GraphSAGE'>, 'GAT': <class 'MolRep.Models.graph_based.GAT.GAT'>, 'GraphNet': <class 'MolRep.Models.graph_based.GraphNet.GraphNet'>, 'MPNN': <class 'MolRep.Models.graph_based.MPNN.MPNN'>, 'CMPNN': <class 'MolRep.Models.graph_based.CMPNN.CMPNN'>, 'DMPNN': <class 'MolRep.Models.graph_based.DMPNN.DMPNN'>, 'MAT': <class 'MolRep.Models.sequence_based.MAT.MAT'>, 'CoMPT': <class 'MolRep.Models.sequence_based.CoMPT.CoMPT'>, 'BiLSTM': <class 'MolRep.Models.sequence_based.BiLSTM.BiLSTM'>, 'SALSTM': <class 'MolRep.Models.sequence_based.SALSTM.SALSTM'>, 'Transformer': <class 'MolRep.Models.sequence_based.Transformer.Transformer'>, 'VAE': <class 'MolRep.Models.unsupervised_based.VAE.VAE'>, 'RandomForest': <class

In [6]:
# export path: model + dataset name
exp_path = os.path.join(output_dir, f'{model_configuration.exp_name}_{dataset_configuration.exp_name}_explainer')

# dataset_config: configuration of the model
# model_name: name of the model
# split_dir: path to defined splits of the dataset
# features_dir: path to data directory
dataset = ExplainerDatasetWrapper(dataset_config=dataset_configuration,
                                  model_name=model_configuration.exp_name,
                                  split_dir=split_dir, features_dir=data_dir)

# set up the experiment
explainer_experiment = ExplainerExperiments(model_configuration, dataset_configuration, exp_path)

In [7]:
import pandas as pd
import re
# import torch

# to load a model from a file without having used it, need to instantiate it first
# model = CMPNN(model_configuration)
# state_dict = torch.load(MODELPATH)
# model.load_state_dict(state_dict)

# Options: IG CAM MCTS GradInput GradCAM RandomBaseline # says RandomBaseLine but code expects 'Random'
ATTRIBUTIONS = []

atom_importances = []
explainer_accuracies = []

if LOADING_FROM_FILES :
    LOGGER_BASE = os.path.join(OUTPUT_DIR, "Logger", f"{DATASET}_explainer")
    logger = Logger(str(os.path.join(LOGGER_BASE, f"{MODEL}_{DATASET}_explainer_by_experiment.log")), mode='a')
    
    ### TODO on first run, uncomment, on subsequent runs can leave commented out ###
    # _, _, _ = explainer_experiment.molecule_importance(dataset=dataset, attribution=ATTRIBUTIONS[0], logger=logger, other={'model_path':MODELPATH}, testing=TESTING)
    
    # read in used explainers
    explainer_accs_path = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer" , f"explainer_accuracies.csv")
    explainer_accuracies_df = pd.read_csv(explainer_accs_path) # used for looking at, nothing else
    
    for e in explainer_accuracies_df['explainer']:
        ATTRIBUTIONS.append(e)
    
    # get atom importances per explainer
    for ATTRIBUTION in ATTRIBUTIONS:
        atom_imp_for_expl = []
        
        imp_path = os.path.join(output_dir, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}_{model_configuration['num_epochs']}_epochs" , "importances.csv")

        # atom importances are given as a string of an array of floats 
        importance = pd.read_csv(imp_path)['Atom_importance']
        
        for _, row in importance.items():
            # remove brackets and newlines
            clean_imp = re.sub(r'[\[\]\n]', '', row).strip()
            
            # split by whitespace and convert to float
            float_imps = [float(x) for x in clean_imp.split()]
            atom_imp_for_expl.append(float_imps)

        atom_importances.append(atom_imp_for_expl)    
    
else: 
    ATTRIBUTIONS = ['IG', 'GradCAM', 'Random']
    # generate explanations for each explainer
    for ATTRIBUTION in ATTRIBUTIONS:
        LOGGER_BASE = os.path.join(OUTPUT_DIR, "Logger", f"{DATASET}_explainer")
        logger = Logger(str(os.path.join(LOGGER_BASE, f"{MODEL}_{DATASET}_explainer_by_{ATTRIBUTION}.log")), mode='a')
        
        # define path for graphics
        svg_dir = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}_{model_configuration['num_epochs']}_epochs")
        os.makedirs(svg_dir, exist_ok=True)
    
        ### comment out if a model exists and want to use it with all models, else trains a new one for each ###
        # explainer_experiment.run_valid(dataset, ATTRIBUTION, logger=logger, other={'model_path':MODELPATH})
    
        ## if model doesn't exist, train a new one
        # if not os.path.exists(MODELPATH):
        #     explainer_experiment.run_valid(dataset, ATTRIBUTION, logger=logger, other={'model_path':MODELPATH})
        
        # generate attributions; bond_importance is None in all the tests I ran
        results, atom_importance, bond_importance = explainer_experiment.molecule_importance(dataset=dataset, attribution=ATTRIBUTION, logger=logger, other={'model_path':MODELPATH}, testing=TESTING)
        
        if DATASET in ['hERG', 'CYP3A4']:
            attribution_results, opt_threshold = explainer_experiment.evaluate_cliffs(dataset, atom_importance, bond_importance)
        else:
            binary = True if ATTRIBUTION == 'MCTS' else False
            #TODO "not working, because ATTRIBUTIONPATH is not set and attributions are not saved"
            attribution_results, opt_threshold = explainer_experiment.evaluate_attributions(dataset, atom_importance, bond_importance, binary=binary)
         
        # creates visualizations from the datasets and outputs them to svg_dir
        explainer_experiment.visualization(dataset, atom_importance, bond_importance, svg_dir=svg_dir, testing=TESTING)
    
        # write the results, atom importances to a file
        df = pd.DataFrame(
            {'SMILES': dataset.get_smiles_list(), 'Atom_importance': atom_importance, 'Bond_importance':bond_importance}
        )
        df.to_csv(os.path.join(svg_dir, "importances.csv"), index=False)
    
        # create an object from the pre-established metrics, add to list (include model results to ensure they're all the same)
        acc = {"explainer" : ATTRIBUTION,
               "model" : MODEL,
                "epochs" : model_configuration['num_epochs'],
                "acc" : results['acc'],
                "auc" : results['auc'], 
                "f1" : results['f1'], 
                "precision" : results['precision'], 
                "recall" : results['recall'], 
                "Attribution AUROC" : attribution_results['Attribution AUROC'], 
                "Attribution F1" : attribution_results['Attribution F1'], 
                "Attribution ACC" : attribution_results['Attribution ACC'], 
                "Attribution Precision" : attribution_results['Attribution Precision'], 
                "Attribution AUROC Mean" : attribution_results['Attribution AUROC Mean'], 
                "Attribution ACC Mean" : attribution_results['Attribution ACC Mean'] }
        explainer_accuracies.append(acc)
    
        # add atom importances to a list, fetch them by index of attributions on the list used
        # importance = {"Atom_importance" : atom_importance}
        atom_importances.append(atom_importance)
       
    # TODO store these in a file
    explainer_accuracies_df = pd.DataFrame(explainer_accuracies)
    
    # dataframe for storing atom importances per explainer in a file later
    importances_df = pd.DataFrame(atom_importances)

In [8]:
import copy

# path to model and explanations

x = os.path.join(output_dir, f"{MODEL}_{DATASET}_explainer", "SVG", f"IG_{model_configuration['num_epochs']}_epochs" , "importances.csv")
# IMPORTANCES_PATH = '../Outputs/20240524_164403/CMPNN_Benzene_explainer/SVG/IG_1_epochs/importances.csv' 

# os.path.join(output_dir, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}_{DATASET}_explainer" , "importances.csv")
IMPORTANCES_PATH = x


# svg_dir = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}_{model_configuration['num_epochs']}_epochs")
# df.to_csv(os.path.join(svg_dir, ), index=False)

# df = pd.read_csv(IMPORTANCES_PATH)
# print(df['Atom_importance'].iat[3])

# TODO if reading importances from a csv, must convert importances from a string of an array of floats to an actual array of floats
# TODO write explainer_accuracies and atom importances to a file if they arent already
# TODO write a method to import them back to notebook

# y = os.path.join(output_dir, f"{MODEL}_{DATASET}_explainer", f"{model_configuration['num_epochs']}_epochs")
# os.makedirs(y, exist_ok=True)

# df.to_csv(os.path.join(svg_dir, "importances.csv"), index=False)

# explainer_accuracies_df.to_csv(y, "explainer_accuracies.csv", index=False)
explainer_accuracies_df

,explainer,model,epochs,acc,auc,f1,precision,recall,Attribution AUROC,Attribution F1,Attribution ACC,Attribution Precision,Attribution AUROC Mean,Attribution ACC Mean
0,IG,CMPNN,1,0.7475,0.84503,0.772693,0.707904,0.850537,0.948686,0.271034,0.826074,0.417503,0.901842,0.822130
1,GradCAM,CMPNN,1,0.7475,0.84503,0.772693,0.707904,0.850537,0.946286,0.231217,0.815562,0.386618,0.893762,0.811085
2,Random,CMPNN,1,0.7475,0.84503,0.772693,0.707904,0.850537,0.495094,0.008616,0.809156,0.037847,0.497988,0.798624


In [9]:
import numpy as np
def convert_from_test_to_real_idx(idx):
    """Converts the index used in the test set to the index in the whole dataset."""    
    smile = dataset.get_smiles_list(testing=True)[idx]
    
    data = dataset.whole_data_df 
    return data.loc[(data == smile).any(axis=1)].index[0]

def convert_real_idx_to_test_idx(idx):
    """Converts an index from the big dataset to the one in the test set"""
    ds = dataset.whole_data_df 
    ts = dataset.get_smiles_list()
    smile = ds.iloc[idx]['SMILES']

    return np.where(ts == smile)[0][0]

In [10]:
def split_molecule(mol_idx:int, explainer='IG', threshold=1e-4, from_whole_data=False):
    """Splits the given molecule into explaining and non-explaining SMILES based on given explanation, returns those and the SMILES of the molecule. Threshold value is the same as in explainerExperiments.py visualization(). from_whole_data True if using idx in the whole dataset, False if looking by index in test set"""
    
    if from_whole_data:
        smile = dataset.whole_data_df.SMILES.iat[mol_idx]
        mol_idx = convert_real_idx_to_test_idx(mol_idx)
    else:
        # testing = True by default
        smile = dataset.get_smiles_list(testing=True)[mol_idx]
    
    # atom importances depend on what explainer method is used, mol_idx is convenient
    expl_idx = ATTRIBUTIONS.index(explainer)
    a_importances = atom_importances[expl_idx][mol_idx]
    
    # print(type(a_importances), a_importances, type(a_importances[0]))
    # print("threshold", threshold, type(threshold))

    original_mol = rdkit.Chem.RWMol(rdkit.Chem.MolFromSmiles(smile))
     
    non_explaining_atom_idxs = []
    explaining_atom_idxs = []
    for idx in range(len(original_mol.GetAtoms())):
        if a_importances[idx] < threshold:
            non_explaining_atom_idxs.append(idx)
        else:
            explaining_atom_idxs.append(idx)
            
    # reverse because removing atoms causes indices of others to shift
    non_explaining_atom_idxs.reverse()
    explaining_atom_idxs.reverse()
    
    # number of atoms in explanation vs whole molecule 
    exp_length = len(explaining_atom_idxs) / ( len(explaining_atom_idxs) + len(non_explaining_atom_idxs))
    
    # find (non)explaining bonds where at least one end is a non-explaining atom
    non_explaining_bonds = []
    explaining_bonds = []
    
    for bond in original_mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        
        # if both end and start in explaining atoms, the edge is explaining 
        if (i and j) not in non_explaining_atom_idxs :
            explaining_bonds.append((i, j))
            explaining_bonds.append((j, i))
        else:
            non_explaining_bonds.append((i, j))
            non_explaining_bonds.append((j, i))
            
            # remove i or j from explaining atoms 
            # so bonds between explaining and nonexplaining atoms are kept in non-explaining molecules
            if i in explaining_atom_idxs: explaining_atom_idxs.remove(i)
            if j in explaining_atom_idxs: explaining_atom_idxs.remove(j)
    
    # explaining molecule has non-explaining bonds and atoms removed
    explaining_mol = copy.deepcopy(original_mol)
    for (i, j) in non_explaining_bonds:
        explaining_mol.RemoveBond(i, j)
    
    for idx in non_explaining_atom_idxs:
        explaining_mol.RemoveAtom(idx)

    explaining_smile = rdkit.Chem.MolToSmiles(explaining_mol)
    
    # comprehensiveness uses non-explaining atoms and edges
    non_explaining_mol = copy.deepcopy(original_mol)
    
    for (i, j) in explaining_bonds:
        non_explaining_mol.RemoveBond(i, j)

    # remove explaining atoms    
    for i in explaining_atom_idxs:
        non_explaining_mol.RemoveAtom(i)
    
    non_explaining_smile = rdkit.Chem.MolToSmiles(non_explaining_mol)
    
    return smile, explaining_smile, non_explaining_smile, exp_length

In [11]:
s, e, n, m = split_molecule(137, from_whole_data=True)

from rdkit import Chem
from rdkit.Chem import Draw

# print(s)
# print(e)
# print(n)
# Draw.MolToImage(Chem.MolFromSmiles(s))
# Draw.MolToImage(Chem.MolFromSmiles(e))
# Draw.MolToImage(Chem.MolFromSmiles(n))
# print(ATTRIBUTION)

In [12]:

# Draw.MolToImage(Chem.MolFromSmiles(n))
# 17 => 93.png

In [13]:
import torch
from MolRep.Featurization.MPNN_embeddings import MolGraph, BatchMolGraph
from rdkit.Chem import MolFromSmiles, rdmolops

def tensors_to_device(smile, batch):
    """Sets nodes, edges, a2b, b2a and adjacency matrix tensors of a molecule to the device"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    adj = rdmolops.GetAdjacencyMatrix(MolFromSmiles(smile))
    
    features = {
        'nodes' : batch.f_atoms, 
        'edges' : batch.f_bonds,
        'a2b' : batch.a2b,
        'b2a' : batch.b2a,
        'adjacency' : torch.FloatTensor(adj) 
    }
    
    for i in features:
        features[i].to(device)
        
    return features['nodes'], features['edges'], features['a2b'], features['b2a'], features['adjacency']

In [14]:
import re
def clean_smile(smile:str) -> str:
    """Takes a (fragmented) SMILES as input and returns concatenated valid fragments of it. Does nothing if input is already valid.
    e.g: C.CC.F.O.n.n -> C.CC.F.O
    """
    cs = ""
    # 1. split n/explanations into a list
    subm = smile.split('.')
    
    # 2. try to convert the pieces into a molgraph
    # if success, append to a new string cs (cut string)
    for (i, sm) in enumerate(subm): 
        try:
            MolGraph(sm)
            cs += sm
            cs += '.'
        except:
            cs += '.'
            continue

    # remove repetitions and the first and last period from the string
    cs = re.sub("\.+", ".", cs)
    # ReGeX from https://stackoverflow.com/a/3331982
    cs = re.sub("\.([^.]*)$", "", cs)
    cs = re.sub("(^\.)", "", cs)
    return cs

In [15]:
def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()

EXPERIMENT_MODEL = MODELPATH

In [19]:
def comp_and_suff(idx, explainer = 'IG', from_all_data=False, modified_formula=True) :
    """Calculates comprehensiveness and sufficiency for a molecule with index idx, given explanation e, returns an object with mol_idx, all the smiles (cleaned and uncleaned), predictions for everything, comprehensiveness and sufficiency, and proportion of explanation vs original molecule. from_all_data True if fetching smile from the whole set, False if from test set."""
    
    # TODO move this out so no need to get the same model every time
    model = explainer_experiment.get_model(dataset, other={'model_path':EXPERIMENT_MODEL})
    model.eval() # set model to evaluation mode

    smile, explaining_smile, non_explaining_smile, exp_length = split_molecule(idx, explainer, from_whole_data=from_all_data)
    
    # clean up the smiles, split may have created invalid molecules
    clean_smiles = list(map(clean_smile, [smile, explaining_smile, non_explaining_smile]))

    preds = []
    for (i, cs) in enumerate(clean_smiles):
        if cs == "": # don't predict empty strings
            # if no non-explaining, then comp is f(g) = great, because the whole molecule explains the prediction!
            # if no explaining, suff is f(g) = maybe good?, because there's nothing to explain why prediction was made?
            
            # none to predict to begin with != none being valid
            if cs == [smile, explaining_smile, non_explaining_smile][i]:
                preds.append([0]) # no (non)explaining to begin with is a-okay     
            else:
                preds.append([None]) # no valid strings after cleaning up less so
            continue
            
        subpreds = []
        
        if modified_formula:
            splits = cs.split('.')
            for split in splits:
            
                # 3. convert s to molgraph
                try:
                    mol = MolGraph(split)
                    g_input = BatchMolGraph( [mol])
                    atoms, bonds, a2b, b2a, adjacency = tensors_to_device(cs, g_input)
            
                    with torch.no_grad():
                        # TODO investigate None params?
                        #   Args for mol2graph:
                        # - mols: A list of SMILES or a list of RDKit molecules.
                        # - atom_descriptors_batch: A list of 2D numpy array containing additional atom descriptors to featurize the molecule
                        subpred = model([[split], None, None]).item()
            
                    subpreds.append(subpred)
                except:
                    # If model couldn't predict the subgraph for whatever weird reason
                    # I don't know why this would happen, but evidently it does happen
                    subpreds.append(None)
        else:
            try:
                mol = MolGraph(cs)
                g_input = BatchMolGraph( [mol])
                atoms, bonds, a2b, b2a, adjacency = tensors_to_device(cs, g_input)
        
                with torch.no_grad():
                    # TODO investigate None params?
                    #   Args for mol2graph:
                    # - mols: A list of SMILES or a list of RDKit molecules.
                    # - atom_descriptors_batch: A list of 2D numpy array containing additional atom descriptors to featurize the molecule
                    subpred = model([[cs], None, None]).item()
        
                subpreds.append(subpred)
            except:
                # If model couldn't predict the subgraph for whatever weird reason
                # I don't know why this would happen, but evidently it does happen
                subpreds.append(None)
            
        preds.append(subpreds)
    
    converted_idx = convert_from_test_to_real_idx(idx)
    
    try:
        mol_pred = preds[0][0] # prediction of original molecule, there is only one so do nothing
        
        if modified_formula: 
            e_pred = None if preds[1] == [None] else sum(preds[1]) / len(preds[1]) # prediction of explaining molecules : take the mean
            n_pred = None if preds[2] == [None] else sum(preds[2]) # prediction of non-explaining molecules : sum
        else:
            e_pred = None if preds[1] == [None] else preds[1][0]
            n_pred = None if preds[2] == [None] else preds[2][0]
        
        # f = model prediction given a smile
        # comp = f(smile) - f(non_explaining_smile)
        # suff = f (smile) - f(explaining_smile)
        
        comp = mol_pred - n_pred if n_pred is not None else None
        suff = mol_pred - e_pred if e_pred is not None else None
        
        result = {
            "explainer" : explainer,
            "mol_idx": idx if from_all_data else converted_idx,
            "smile" : smile,
            "explaining_smile" : explaining_smile,
            "clean_explanation" : clean_smiles[1],
            "non_explaining_smile" : non_explaining_smile,
            "clean_nonexplanation": clean_smiles[2],
            "mol_pred": mol_pred,
            "e_pred": e_pred,
            "ne_pred": n_pred,
            "comp" : comp,
            "suff" : suff,
            "expl length": exp_length
        }
        
        return result
    except:
        # print 
        print(preds, " idx in dataset: ", converted_idx , " idx in test set: ", idx)


In [20]:
van_3 = comp_and_suff(3, modified_formula=False, from_all_data=True) # mol 3 in whole set
mod_3 = comp_and_suff(3, from_all_data=True)

van_509 = comp_and_suff(509, modified_formula=False, from_all_data=True)
mod_509 = comp_and_suff(509, from_all_data=True) 

# print(split_molecule(3, from_whole_data=False)) # mol idx is number 3 in test set i.e. mol 13
# print(split_molecule(3, from_whole_data=True)) # mol idx is number 3 in whole set
# Draw.MolToImage()

Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda


In [21]:
c = pd.DataFrame([van_3, mod_3, van_509, mod_509])
c
# print(van_3)
# print(mod_3)

,explainer,mol_idx,smile,explaining_smile,clean_explanation,non_explaining_smile,clean_nonexplanation,mol_pred,e_pred,ne_pred,comp,suff,expl length
0,IG,3,CC(=O)N(C)c1ccc(c2c1cccc2)[N+](=O)[O-],O=CNc1ccc([N+](=O)[O-])c2ccccc12,O=CNc1ccc([N+](=O)[O-])c2ccccc12,CC.CN,CC.CN,0.565568,0.572062,0.453750,0.111818,-0.006494,0.888889
1,IG,3,CC(=O)N(C)c1ccc(c2c1cccc2)[N+](=O)[O-],O=CNc1ccc([N+](=O)[O-])c2ccccc12,O=CNc1ccc([N+](=O)[O-])c2ccccc12,CC.CN,CC.CN,0.565568,0.572062,0.922270,-0.356702,-0.006494,0.888889
2,IG,509,C[C@@H]1C[C@H]1NC(=O)N1CCc2cc(ccc2C1)F,NC=O.c1ccccc1,NC=O.c1ccccc1,CCN(C)Cc.C[C@@H]1CC1.cF,C[C@@H]1CC1,0.509846,0.567338,0.438039,0.071808,-0.057492,0.500000
3,IG,509,C[C@@H]1C[C@H]1NC(=O)N1CCc2cc(ccc2C1)F,NC=O.c1ccccc1,NC=O.c1ccccc1,CCN(C)Cc.C[C@@H]1CC1.cF,C[C@@H]1CC1,0.509846,0.535968,0.438039,0.071808,-0.026122,0.500000


In [136]:
def comp_and_suff_data(explainer='IG', sample_size=2400):
    """Accumulates the comp and suff for all molecules within the test set of the provided dataset."""
    c_and_s = []
    
    n = min(sample_size, len(dataset.get_smiles_idxs(testing=True)))

    # we only care about the index, because we're getting things by index in the test set
    for i in range(n):
        torch.cuda.empty_cache() # please be enough to allow things to run smoothly
        vals = comp_and_suff(i, explainer)
        c_and_s.append(vals)
    return c_and_s


# c = comp_and_suff_data(sample_size=10)
# pd.DataFrame(c)

In [137]:
# TODO 2 further work
# save results of comp-suff for transparency
# use a different explainer on the model
# save those results

def comp_and_suff_for_all_explainers(explainers, sample_size=100):
    """Calculates comp and suff for all explainers, given a list of explainers and a full matrix of atom importances."""
    comps = []
    
    for (i, explainer) in enumerate(explainers):
        torch.cuda.empty_cache() # please be enough to allow things to run smoothly
        # print(i, explainer)
        
        comp = comp_and_suff_data(explainer=explainer, sample_size=sample_size)
        
        comps.append(comp)
    return comps

c_and_s = comp_and_suff_for_all_explainers(ATTRIBUTIONS, sample_size=100)
# c_and_s = comp_and_suff_data(dataset, 'IG', sample_size=60)

Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model to cuda
Moving model 

In [146]:
pd.DataFrame(c_and_s[0])

,explainer,mol_idx,smile,explaining_smile,clean_explanation,non_explaining_smile,clean_nonexplanation,mol_pred,e_pred,ne_pred,comp,suff,expl length
0,IG,3,CC(=O)N(C)c1ccc(c2c1cccc2)[N+](=O)[O-],O=CNc1ccc([N+](=O)[O-])c2ccccc12,O=CNc1ccc([N+](=O)[O-])c2ccccc12,CC.CN,CC,0.565568,0.572062,0.459984,0.105584,-0.006494,0.888889
1,IG,9,COC[C@H](C(=O)N)NC(=O)Cc1cccc(c1)F,CN.CN.c1ccccc1,CN.CN,C=O.CC=O.CCOC.cF,C=O.CC=O.CCOC,0.518624,0.462286,1.397677,-0.879052,0.056338,0.555556
2,IG,10,c1cc(cc(c1)F)CNc1ccc(cc1)C(=O)NC1CCOCC1,NCc1ccc(N)cc1.c1ccccc1,NCc1ccc(N)cc1,C=O.NC1CCOCC1.cC.cF,C=O.NC1CCOCC1,0.540191,0.562552,0.907459,-0.367268,-0.022361,0.625000
3,IG,13,Cn1cncc1CCNC(=O)c1cc(cnc1)F,cccc(c)C(N)=O.ccnc,,Cn.cCC.cF.cn.cn,,0.546673,NaN,NaN,NaN,NaN,0.666667
4,IG,17,CN(CC[C@@H](c1ccccc1)O)C(=O)[C@@H]1CC[N@H+](CC...,C.CN.c1ccccc1,C.CN,C=O.CNCCCO.C[N@H+]1CC[C@H](C=O)CC1,C=O.CNCCCO,0.489150,0.467179,0.917614,-0.428464,0.021971,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,IG,496,C[C@@H](c1ccc(cc1)n1cncn1)NC(=O)NCCN1CCOCC1,NCN.c.cn-c1ccccc1,NCN,C=O.CC.NCCN1CCOCC1.cn.cn,C=O.CC.NCCN1CCOCC1,0.517687,0.454082,1.367905,-0.850218,0.063605,0.480000
96,IG,497,Cn1c(nnc1[C@H]1CCC[N@H+](C1)C1CCCCC1)Cn1ccnc1,ccnc.cnc,,Ccnn.Cn.c[C@H]1CCC[N@@H+](C2CCCCC2)C1.cn,,0.475073,NaN,NaN,NaN,NaN,0.291667
97,IG,498,Cc1ncc(s1)[C@@H](C)[NH2+][C@H](C)CCSC,c.cc,,CSCC[C@@H](C)[NH2+][C@H](C)cs.Ccn,,0.476645,NaN,NaN,NaN,NaN,0.200000
98,IG,506,CCOC(=O)[C@@H]1C[N@@H+](C[C@@H]1C(=O)c1ccccc1)C,C.Cc1ccccc1,C,C=O.CCO.C[N@@H+]1CC[C@H](C=O)C1,C=O.CCO,0.504132,0.472073,0.927723,-0.423591,0.032059,0.421053


In [148]:
pd.DataFrame(c_and_s[1])

,explainer,mol_idx,smile,explaining_smile,clean_explanation,non_explaining_smile,clean_nonexplanation,mol_pred,e_pred,ne_pred,comp,suff,expl length
0,GradCAM,3,CC(=O)N(C)c1ccc(c2c1cccc2)[N+](=O)[O-],O=CNc1ccc([N+](=O)[O-])c2ccccc12,O=CNc1ccc([N+](=O)[O-])c2ccccc12,CC.CN,CC,0.565568,0.572062,0.459984,0.105584,-0.006494,0.888889
1,GradCAM,9,COC[C@H](C(=O)N)NC(=O)Cc1cccc(c1)F,NC=O.NC=O.c1ccccc1,NC=O.NC=O,CC.CCOC.cF,CC.CCOC,0.518624,0.490961,0.906409,-0.387785,0.027663,0.666667
2,GradCAM,10,c1cc(cc(c1)F)CNc1ccc(cc1)C(=O)NC1CCOCC1,NC(=O)c1ccc(N)cc1.c1ccccc1,NC(=O)c1ccc(N)cc1,NC1CCOCC1.cC.cF,NC1CCOCC1,0.540191,0.567293,0.431738,0.108453,-0.027102,0.666667
3,GradCAM,13,Cn1cncc1CCNC(=O)c1cc(cnc1)F,cccc(c)C(N)=O.ccnc,,Cn.cCC.cF.cn.cn,,0.546673,NaN,NaN,NaN,NaN,0.666667
4,GradCAM,17,CN(CC[C@@H](c1ccccc1)O)C(=O)[C@@H]1CC[N@H+](CC...,NC=O.NC=O.O.c1ccccc1,NC=O.NC=O.O,CCCNC.C[C@H]1CC[N@H+](C)CC1,CCCNC,0.489150,0.484110,0.443845,0.045306,0.005041,0.541667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,GradCAM,496,C[C@@H](c1ccc(cc1)n1cncn1)NC(=O)NCCN1CCOCC1,NC(N)=O.c.cn-c1ccccc1,NC(N)=O,CC.NCCN1CCOCC1.cn.cn,CC.NCCN1CCOCC1,0.517687,0.490683,0.892184,-0.374497,0.027004,0.520000
96,GradCAM,497,Cn1c(nnc1[C@H]1CCC[N@H+](C1)C1CCCCC1)Cn1ccnc1,ccnc.cnc,,Ccnn.Cn.c[C@H]1CCC[N@@H+](C2CCCCC2)C1.cn,,0.475073,NaN,NaN,NaN,NaN,0.291667
97,GradCAM,498,Cc1ncc(s1)[C@@H](C)[NH2+][C@H](C)CCSC,c.cc,,CSCC[C@@H](C)[NH2+][C@H](C)cs.Ccn,,0.476645,NaN,NaN,NaN,NaN,0.200000
98,GradCAM,506,CCOC(=O)[C@@H]1C[N@@H+](C[C@@H]1C(=O)c1ccccc1)C,C=O.O=Cc1ccccc1,C=O,CCO.C[C@H]1CC[N@@H+](C)C1,CCO,0.504132,0.475721,0.452002,0.052130,0.028411,0.526316


In [149]:
pd.DataFrame(c_and_s[2])

,explainer,mol_idx,smile,explaining_smile,clean_explanation,non_explaining_smile,clean_nonexplanation,mol_pred,e_pred,ne_pred,comp,suff,expl length
0,Random,3,CC(=O)N(C)c1ccc(c2c1cccc2)[N+](=O)[O-],CC(=O)N(C)c1ccc([N+](=O)[O-])c2ccccc12,CC(=O)N(C)c1ccc([N+](=O)[O-])c2ccccc12,,,0.565568,0.563076,0.000000,0.565568,0.002492,1.000000
1,Random,9,COC[C@H](C(=O)N)NC(=O)Cc1cccc(c1)F,COC[C@H](CN)NC(=O)Cc1cccc(F)c1,COC[C@H](CN)NC(=O)Cc1cccc(F)c1,C=O,C=O,0.518624,0.511699,0.475721,0.042903,0.006926,0.944444
2,Random,10,c1cc(cc(c1)F)CNc1ccc(cc1)C(=O)NC1CCOCC1,O=C(NC1CCOCC1)c1ccc(NCc2cccc(F)c2)cc1,O=C(NC1CCOCC1)c1ccc(NCc2cccc(F)c2)cc1,,,0.540191,0.538274,0.000000,0.540191,0.001917,1.000000
3,Random,13,Cn1cncc1CCNC(=O)c1cc(cnc1)F,Cn1cncc1CCNC(=O)c1cncc(F)c1,Cn1cncc1CCNC(=O)c1cncc(F)c1,,,0.546673,0.545323,0.000000,0.546673,0.001350,1.000000
4,Random,17,CN(CC[C@@H](c1ccccc1)O)C(=O)[C@@H]1CC[N@H+](CC...,CN(CC[C@H](O)c1ccccc1)C(=O)[C@H]1CC[N@H+](CC(N...,CN(CC[C@H](O)c1ccccc1)C(=O)[C@H]1CC[N@H+](CC(N...,,,0.489150,0.490564,0.000000,0.489150,-0.001413,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Random,496,C[C@@H](c1ccc(cc1)n1cncn1)NC(=O)NCCN1CCOCC1,C[C@H](NC(=O)NCCN1CCOCC1)c1ccc(-n2cncn2)cc1,C[C@H](NC(=O)NCCN1CCOCC1)c1ccc(-n2cncn2)cc1,,,0.517687,0.512078,0.000000,0.517687,0.005609,1.000000
96,Random,497,Cn1c(nnc1[C@H]1CCC[N@H+](C1)C1CCCCC1)Cn1ccnc1,Cn1c(Cn2ccnc2)nnc1[C@H]1CCC[N@@H+](C2CCCCC2)C1,Cn1c(Cn2ccnc2)nnc1[C@H]1CCC[N@@H+](C2CCCCC2)C1,,,0.475073,0.481153,0.000000,0.475073,-0.006080,1.000000
97,Random,498,Cc1ncc(s1)[C@@H](C)[NH2+][C@H](C)CCSC,CSCC[C@@H](C)[NH2+][C@H](C)c1cnc(C)s1,CSCC[C@@H](C)[NH2+][C@H](C)c1cnc(C)s1,,,0.476645,0.475088,0.000000,0.476645,0.001557,1.000000
98,Random,506,CCOC(=O)[C@@H]1C[N@@H+](C[C@@H]1C(=O)c1ccccc1)C,CCOC(=O)[C@@H]1C[N@H+](C)C[C@@H]1C(=O)c1ccccc1,CCOC(=O)[C@@H]1C[N@H+](C)C[C@@H]1C(=O)c1ccccc1,,,0.504132,0.500708,0.000000,0.504132,0.003424,1.000000


In [138]:
clear_gpu_memory()

# Cc1ccc(cc1)c1nc(c(nn1)C)Sc1cccc(c1)C(F)(F)F
# c = comp_and_suff(137, convert_idx=False)
# print(c)
# 
# s , e, n, _ = split_molecule(137, convert_idx=False)
# print(s, e, n, convert_from_test_to_real_idx(137))
# clean_smile(n)
# print(convert_real_idx_to_test_idx(11815))
# s, e, n, m = split_molecule(11815, explainer='IG',convert_idx=True)
# print(s, m)

# Draw.MolToImage(MolFromSmiles(s))

In [409]:
def avg_comp_and_suff(d):
    """Calculates the average comp and suff for a list of data. Returns the original data length, avg comp and suff, and how many samples were used to calculate them."""

    c, n, s, m, l = 0, 0, 0, 0, 0
    
    for i in range(len(d)):
        item = d[i]
        mol_pred = item.get('mol_pred')
        e_pred = item.get('e_pred')
        ne_pred = item.get('ne_pred')
        lgt = item.get('expl length')
        
        l += lgt if lgt is not None else 0
        
        # print(i, mol_pred, e_pred, ne_pred)

        if ne_pred is not None:
            # proportion of comprehensiveness to mol prediction
            c += (abs (item.get('comp') / mol_pred) )
            n += 1

        if e_pred is not None:
            # absolute value of sufficiency
            s += abs(item.get('suff'))
            m += 1

    # average out the results
    avg_comp = c/n if n != 0 else None
    avg_suff = s/m if m != 0 else None
    avg_length = l / m

    results = { "explainer" : d[0]['explainer'],
                "original samples" : len(d),
                "average comp %" : avg_comp,
                "average suff" : avg_suff,
                "samples for avg comp" : n,
                "samples for avg suff" : m,
                "avg expl length": avg_length}
    
    return results

In [410]:
# print(c_and_s[0])

avgs = []
for c in c_and_s:
    a = avg_comp_and_suff(c)
    avgs.append(a)

pd.DataFrame(avgs)

,explainer,original samples,average comp %,average suff,samples for avg comp,samples for avg suff,avg expl length
0,IG,100,0.882992,0.045042,83,85,0.505371
1,GradCAM,100,0.624151,0.041177,82,90,0.574239
2,Random,100,0.990827,0.002129,100,100,0.999444


In [478]:
# print(dataset.get_smiles_list().index())
# print(df.loc[df == 'CC(=O)N(C)c1ccc(c2c1cccc2)[N+](=O)[O-]'])
c = comp_and_suff(0)

Moving model to cuda


In [502]:
i = convert_real_idx_to_test_idx(509)
d = comp_and_suff(i, from_all_data=False)
d = comp_and_suff(509, from_all_data=True)


Moving model to cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 

In [480]:
pd.DataFrame([c, d])

,explainer,mol_idx,smile,explaining_smile,clean_explanation,non_explaining_smile,clean_nonexplanation,mol_pred,e_pred,ne_pred,comp,suff,expl length
0,IG,3,CC(=O)N(C)c1ccc(c2c1cccc2)[N+](=O)[O-],O=CNc1ccc([N+](=O)[O-])c2ccccc12,O=CNc1ccc([N+](=O)[O-])c2ccccc12,CC.CN,CC.CN,0.565568,0.572062,0.922270,-0.356702,-0.006494,0.888889
1,IG,509,C[C@@H]1C[C@H]1NC(=O)N1CCc2cc(ccc2C1)F,NC=O.c1ccccc1,NC=O.c1ccccc1,CCN(C)Cc.C[C@@H]1CC1.cF,C[C@@H]1CC1,0.509846,0.535968,0.438039,0.071808,-0.026122,0.500000
